Sources: https://towardsdatascience.com/web-scraping-news-articles-in-python-9dd605799558 https://jamesrledoux.com/code/drop_duplicates https://github.com/CWTSLeiden/cwts_covid https://stackoverflow.com/questions/10715965/add-one-row-to-pandas-dataframe https://www.geeksforgeeks.org/python-pandas-dataframe-append/ https://www.geeksforgeeks.org/create-a-list-from-rows-in-pandas-dataframe/ https://www.guru99.com/python-list-count.html

In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import time

%matplotlib inline
import matplotlib.pyplot as plt

# magics and warnings
%load_ext autoreload
%autoreload 2
import warnings; warnings.simplefilter('ignore')

import os, random, codecs, json
import pandas as pd
import numpy as np

seed = 99
random.seed(seed)
np.random.seed(seed)

import nltk, sklearn
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="white")
sns.set_context("notebook", font_scale=1.2, rc={"lines.linewidth": 2.5})

In [2]:
# load Altmetrics data

# separately export twitter IDs if necessary (to hydrate them)

out_folder = "datasets_output/json_altmetrics_cwts"
all_altmetrics = []

for root, dirs, files in os.walk(out_folder):
    for file in files:
        if ".json" in file:
            all_altmetrics.append(json.loads(codecs.open(os.path.join(root,file)).read()))

In [3]:
df_news = pd.DataFrame.from_dict({
    "first_seen_on":[r["citation"]["first_seen_on"] if "first_seen_on" in r["citation"] else "" for r in all_altmetrics],
    "counts_news":[r["counts"]["news"]["posts_count"] if "news" in r["counts"].keys() else 0 for r in all_altmetrics],
    "news_journals":[r["counts"]["news"]["unique_users"] if "news" in r["counts"].keys() else 0 for r in all_altmetrics],
    "doi":[r["citation"]["doi"] if "doi" in r["citation"] else "" for r in all_altmetrics],
})

df_news['year'] = pd.DatetimeIndex(df_news["first_seen_on"]).year

df_news = df_news[df_news.year >= 2020.0]
df_news = df_news[df_news.counts_news > 0]

df_news.shape

(1675, 5)

In [4]:
# https://www.geeksforgeeks.org/create-a-list-from-rows-in-pandas-dataframe/

# Select the top journals

# Create an empty list 
Row_list =[] 
sort_journals = []
  
# Iterate over each row 
for index, rows in df_news.iterrows(): 
    # Create list for the current row 
    my_list = rows.news_journals
    
    # append the list to the final list 
    Row_list.extend(my_list) 

df_journals = pd.DataFrame(columns=["journal", "counts"])
    
for i in range(len(Row_list)):
    count = Row_list.count(Row_list[i])
    #print(Row_list[i], '=', count)
    df_journals.loc[i] = [Row_list[i]] + [count]
    
#print(Row_list) 

In [19]:
df_journals = df_journals.drop_duplicates()

df_journals.sort_values(by=['counts'])[2070:2128]

,journal,counts
369,newsbeezer,71
407,seattle_post_intelligencer,71
421,idaho_press,71
410,new_canaan_advertiser,71
412,the_darien_times,71
423,goskagit_com,72
244,science_daily,72
550,gizmodo_australia,73
1022,daily_mail,73
61,medical_news_today,73


In [6]:
#change journal here to selecte DOI's with news article from other journal
journal = 'yahoo'

In [7]:
# Pick a specific journal and create Altemtrics API link

API_URL_list = []
dois_list = []
journal_doi = []
length = df_news.shape[0]
print(length)
url1 = 'https://api.altmetric.com/v1/fetch/doi/'
url2 = '?key=bfc9115e2490100d074ded6a3a2cb845'

for i in range(1675):
    for element in df_news.news_journals[i-1:i]:
        if journal in element:
            for entry in df_news.doi[i-1:i]:
                if df_news.doi[i-1:i] is not None:
                    journal_doi.append(journal + ' ' + entry)
                    dois_list.append(entry)
                    API_URL_list.append(url1 + entry + url2)

print(API_URL_list)

1675
['https://api.altmetric.com/v1/fetch/doi/10.2139/ssrn.3561244?key=bfc9115e2490100d074ded6a3a2cb845', 'https://api.altmetric.com/v1/fetch/doi/10.1186/s13578-020-00404-4?key=bfc9115e2490100d074ded6a3a2cb845', 'https://api.altmetric.com/v1/fetch/doi/10.1101/2020.03.11.20031096?key=bfc9115e2490100d074ded6a3a2cb845', 'https://api.altmetric.com/v1/fetch/doi/10.1038/s41591-020-0819-2?key=bfc9115e2490100d074ded6a3a2cb845', 'https://api.altmetric.com/v1/fetch/doi/10.1101/2020.04.23.20076042?key=bfc9115e2490100d074ded6a3a2cb845', 'https://api.altmetric.com/v1/fetch/doi/10.1016/s0140-6736(20)30753-4?key=bfc9115e2490100d074ded6a3a2cb845', 'https://api.altmetric.com/v1/fetch/doi/10.1056/nejmc2001899?key=bfc9115e2490100d074ded6a3a2cb845', 'https://api.altmetric.com/v1/fetch/doi/10.1016/j.kint.2020.03.001?key=bfc9115e2490100d074ded6a3a2cb845', 'https://api.altmetric.com/v1/fetch/doi/10.1016/j.meegid.2020.104351?key=bfc9115e2490100d074ded6a3a2cb845', 'https://api.altmetric.com/v1/fetch/doi/10.732

In [8]:
import json
import requests
from bs4 import BeautifulSoup

def get_json_from_page(url):
    res = requests.get(url).json()
    #html_page = res.content

    #soup = BeautifulSoup(html_page, "lxml")

    #dictionary = json.loads(res)
    return res

def print_all_news(dictionary):
    for news_data in dictionary['posts']['news']:
        url = news_data['url']
        print(url)
        print(get_thread(url))
        
def get_thread(url):
    res = requests.get(url)
    html_page = res.content

    soup = BeautifulSoup(html_page, 'html.parser')

    items = soup.findAll('p', {'class': 'TweetTextSize'})
    all_text = ""
    for item in items:
        print(item.getText())